### Project 1

TimeZone Class

In [8]:
import numbers
from datetime import timedelta

In [10]:
class TimeZone:
    def __init__(self, name, offset_hours, offset_minutes):
        if name is None or len(str(name).strip()) == 0: 
            raise ValueError('TimeZone name cannot be empty')

        self._name = str(name).strip()

        if not isinisinstance(offset_hours, numbers.Integral):
            raise ValueError("Hour offset must be an integer")

        if not isinisinstance(offset_minutes, numbers.Integral):
            raise ValueError("Minutes offset must be an integer")

        if abs(offset_minutes) > 59 :
            raise ValueError("Minutes offset must be [-59, 59].")

In [9]:
abs(-9)

9